Set up env

In [1]:
import carla
import numpy as np
from src.simulator import Simulator
from src.agent import NCPAgent
from src.model import Model, Trainer

import torchvision
from IPython.display import display
import sys
import os
import torch
sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent


# Training process

In [2]:
from src.model import Model, Trainer

In [4]:
simulator = Simulator(world_name='Town03_opt')

In [5]:
import time

simulator.world.get_spectator().set_transform(
    carla.Transform(
        location=carla.Location(x=398.7934265136719,
                                y=-56.03200912475586,
                                z=3.37939715385437)))

simulator.spawn_car_with_camera(
    rel_coordinates=carla.Location(x=1.2, z=1.9) # camera coords
)
vehicle = simulator.get_vehicle()


output_size = 1
units = 19
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

ncp = Model(output_size, units)
ncp.to(device)
if not os.path.isdir(f'out/{simulator.world_name}'):
    os.mkdir(f'out/{simulator.world_name}')
with open(f'out/{simulator.world_name}/data.txt', 'a+') as f:
    f.write(f'timestamp start = {time.time()}\n')
agent = NCPAgent(simulator, ncp, target_speed=10)

next_waypoint  = [simulator.world.get_map().get_waypoint(vehicle.get_location(),
                                                    project_to_road=True,
                                                    lane_type=(carla.LaneType.Driving))]

waypoints = []
dist_between_waypoints = 20
waypoint_num = 350
# waypoint_num = 10
for _ in range(waypoint_num):
    waypoints.append(next_waypoint[-1])
    # simulator.world.get_spectator().set_transform(next_waypoint[-1].transform)
    next_waypoint = next_waypoint[-1].next(dist_between_waypoints)

dest_idx = 2
dest = waypoints[dest_idx].transform.location
agent.set_destination(dest)
agent.set_target_speed(10)

cuda
alloc!
Camera stream started


In [6]:
# os.rmdir('./out/')

idx = 1
waypoint = waypoints[idx]
to_PIL = torchvision.transforms.ToPILImage()

loss_func = torch.nn.functional.mse_loss
optimizer = torch.optim.Adam(ncp.parameters(), lr=0.001)

# basic_agent = BasicAgent(vehicle=vehicle)
trainer = Trainer(ncp, loss_func, optimizer)

while True:
    # if vehicle.get_location().distance(dest) < 0.5:
    #     print(f"Destination reached")
    #     break
    # if (vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints / 4) or \
    #     (vehicle.get_location().distance(waypoint.transform.location) >= dist_between_waypoints * 2 and \
    #      vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints * 3):
    #     print(f'Waypoint {idx} was reached')
    #     waypoint = waypoints[idx + 1]
    #     idx += 1
    
    control, movement, raw_data, out_tensor = agent.run_step()
    # if raw_data is not None:
    #     trainer.train(raw_data, torch.tensor(control.steer))
    #     print(out_tensor)
    vehicle.apply_control(control)
    if agent.simulator.image_frame is not None:
        with open(f'out/{simulator.world_name}/data.txt', 'a+') as f:
            f.write(f'{agent.simulator.image_frame} : {control.steer}\n')

    
    if agent.done():
        if dest_idx < waypoint_num - 1:
            dest_idx += 10
            dest_idx = min(dest_idx, waypoint_num - 1)
            print(f'Intermediate destination reached. Moving to waypoint {dest_idx}')
            agent.is_done = False
            agent.set_destination(waypoints[dest_idx].transform.location)
            continue

        print("The target has been reached, stopping the simulation")
        break
vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))

Intermediate destination reached. Moving to waypoint 12
Intermediate destination reached. Moving to waypoint 22
Intermediate destination reached. Moving to waypoint 32
Intermediate destination reached. Moving to waypoint 42
Intermediate destination reached. Moving to waypoint 52
Intermediate destination reached. Moving to waypoint 62
Intermediate destination reached. Moving to waypoint 72
Intermediate destination reached. Moving to waypoint 82
Intermediate destination reached. Moving to waypoint 92
Intermediate destination reached. Moving to waypoint 102
Intermediate destination reached. Moving to waypoint 112
Intermediate destination reached. Moving to waypoint 122
Intermediate destination reached. Moving to waypoint 132
Intermediate destination reached. Moving to waypoint 142
Intermediate destination reached. Moving to waypoint 152
Intermediate destination reached. Moving to waypoint 162
Intermediate destination reached. Moving to waypoint 172
Intermediate destination reached. Moving

In [7]:
simulator.destroy_all()

Camera stream stopped


In [ ]:
# ncp.save_model(f'./model/pretrained_wp{waypoint_num}.pth')

In [ ]:
simulator.destroy_all()


# test

In [ ]:
ncp.load_state_dict(torch.load(f'model/pretrained_wp{waypoint_num}.pth'))

In [ ]:
print(ncp)

In [ ]:
ncp.eval()

In [ ]:
next_waypoint  = [simulator.world.get_map().get_waypoint(vehicle.get_location(),
                                                    project_to_road=True,
                                                    lane_type=(carla.LaneType.Driving))]

waypoints = []
dist_between_waypoints = 20
waypoint_num = 5
for _ in range(waypoint_num):
    waypoints.append(next_waypoint[-1])
    # simulator.world.get_spectator().set_transform(next_waypoint[-1].transform)
    next_waypoint = next_waypoint[-1].next(dist_between_waypoints)

dest_idx = 2
dest = waypoints[dest_idx].transform.location
agent.set_destination(dest)
agent.set_target_speed(10)

In [ ]:
idx = 1
waypoint = waypoints[idx]
to_PIL = torchvision.transforms.ToPILImage()

loss_func = torch.nn.functional.mse_loss
optimizer = torch.optim.Adam(ncp.parameters(), lr=0.001)

# basic_agent = BasicAgent(vehicle=vehicle)
# trainer = Trainer(ncp, loss_func, optimizer)

while True:
    # if vehicle.get_location().distance(dest) < 0.5:
    #     print(f"Destination reached")
    #     break
    # if (vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints / 4) or \
    #     (vehicle.get_location().distance(waypoint.transform.location) >= dist_between_waypoints * 2 and \
    #      vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints * 3):
    #     print(f'Waypoint {idx} was reached')
    #     waypoint = waypoints[idx + 1]
    #     idx += 1
    
    control, movement, raw_data, out_tensor = agent.run_step()
    # trainer.train(raw_data, torch.tensor(control.steer))
    print(control.steer, movement.item(), out_tensor)
    new_control = carla.VehicleControl(steer=movement.item(), throttle=control.throttle, brake=control.brake)
    vehicle.apply_control(new_control)
    
    if agent.done():
        if dest_idx < waypoint_num - 1:
            dest_idx += 10
            dest_idx = min(dest_idx, waypoint_num - 1)
            print(f'Intermediate destination reached. Moving to waypoint {dest_idx}')
            agent.is_done = False
            agent.set_destination(waypoints[dest_idx].transform.location)
            continue

        print("The target has been reached, stopping the simulation")
        break
vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))

In [ ]:
simulator.destroy_all()

In [ ]:
# # we will save the conv layer weights in this list
# model_weights =[]
# #we will save the 49 conv layers in this list
# conv_layers = []
# # get all the model children as list
# model_children = list(model.children())
# #counter to keep count of the conv layers
# counter = 0
# #append all the conv layers and their respective wights to the list
# for i in range(len(model_children)):
#     if type(model_children[i]) == nn.Conv2d:
#         counter+=1
#         model_weights.append(model_children[i].weight)
#         conv_layers.append(model_children[i])
#     elif type(model_children[i]) == nn.Sequential:
#         for j in range(len(model_children[i])):
#             for child in model_children[i][j].children():
#                 if type(child) == nn.Conv2d:
#                     counter+=1
#                     model_weights.append(child.weight)
#                     conv_layers.append(child)
# print(f"Total convolution layers: {counter}")
# print(f"{conv_layers}")

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)

In [ ]:
# image = transform(image)
# image = image[:3,:,:]
# print(f"Image shape before: {image.shape}")
# image = image.unsqueeze(0)
# print(f"Image shape after: {image.shape}")
# image = image.to(device)

In [ ]:
# outputs = []
# names = []
# for layer in conv_layers[0:]:
#     image = layer(image)
#     outputs.append(image)
#     names.append(str(layer))
# print(len(outputs))
# # print feature_maps
# for feature_map in outputs:
#     print(feature_map.shape)

In [ ]:
# processed = []
# for feature_map in outputs:
#     feature_map = feature_map.squeeze(0)
#     gray_scale = torch.sum(feature_map,0)
#     gray_scale = gray_scale / feature_map.shape[0]
#     processed.append(gray_scale.data.cpu().numpy())
# for fm in processed:
#     print(fm.shape)


In [ ]:
# fig = plt.figure(figsize=(30, 50))
# for i in range(len(processed)):
#     a = fig.add_subplot(5, 4, i+1)
#     imgplot = plt.imshow(processed[i])
#     a.axis("off")
#     a.set_title(names[i].split('(')[0], fontsize=30)
# plt.savefig(str('feature_maps.jpg'), bbox_inches='tight')


In [ ]:
# import torch
# import torch.nn as nn
# import pytorch_lightning as pl
# import torch.utils.data as data

# from src.model import SequenceLearner, NCP

# ncp = NCP(image_features.shape[1], output_size, units)
# ncp_model = ncp.get_ncp()
# learner = SequenceLearner(ncp_model, lr=0.01)
# trainer = pl.Trainer(
#     logger=pl.loggers.CSVLogger("log"),
#     max_epochs=5,
#     gradient_clip_val=1, #clip grad for training stabilizing
# )

Agent test

In [ ]:
import glob, os, sys
# sys.path.append(glob.glob('CARLA_SIM/PythonAPI/carla/'))
sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent
# from agents.navigation.behavior_agent import BehaviorAgent

In [ ]:
simulator.spawn_car_with_camera()

In [ ]:
vehicle = simulator.get_vehicle()

In [ ]:
# agent = BehaviorAgent(vehicle, behavior='aggressive')
agent = BasicAgent(vehicle)
dest = simulator.spawn_points[50].location
agent.set_destination(dest)
agent.follow_speed_limits(False)
agent.set_target_speed(90)

In [ ]:
import time
while True:
    if agent.done():
        print("The target has been reached, stopping the simulation")
        break
    # if collided == True:
    #     vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))
    #     print("Collision detected. Abort")
    #     break

    vehicle.apply_control(agent.run_step())
    # time.sleep(0.5)

In [ ]:
simulator.destroy_all()

In [7]:
from src.model import TrainingDataset

td = TrainingDataset(annotations_file='out/Town01_opt/data.txt',
                     img_dir='out/Town01_opt')

# photos = td.labels[0].unique()
# photo_and_angle = {}
# for photo in photos:
#     td.labels.sum()
td_unique = td.labels.groupby(by=['Image']).Steer_angle.mean().reset_index()
td_unique

         Image  Steer_angle
0         1426    -0.274289
1         1426    -0.274289
2         1426    -0.274289
3         1426    -0.274289
4         1426    -0.274289
...        ...          ...
777489  207211     0.000023
777490  207211     0.000020
777491  207211     0.000022
777492  207211     0.000022
777493  207212     0.000000

[777494 rows x 2 columns]


,Image,Steer_angle
0,1426,-0.274289
1,1443,-0.274289
2,1444,-0.274289
3,1445,-0.274289
4,1446,-0.274289
...,...,...
203924,207208,0.000025
203925,207209,0.000024
203926,207210,0.000022
203927,207211,0.000022


In [1]:
from src.model import Trainer, Model
import torch

ncp = Model(3, 19)
loss_func = torch.nn.functional.mse_loss
optimizer = torch.optim.Adam(ncp.parameters(), lr=0.001)

tr = Trainer(model=ncp,
             loss_func=loss_func,
             optimizer=optimizer,
             annotations_file='out/test_data/data.txt',
             img_dir='out/test_data',
             test_size=0.4)

alloc!


In [2]:
tr.device

'cpu'

In [3]:
tr.train(epochs=1)

Epoch 0
[(tensor([[[243, 249, 210,  ..., 241, 245, 232],
         [231, 238, 203,  ..., 235, 246, 180],
         [206, 221, 167,  ..., 217, 209, 196],
         ...,
         [128, 113, 108,  ..., 114, 119, 134],
         [129, 142, 116,  ..., 129, 140, 133],
         [104, 113, 109,  ..., 147, 132, 142]],

        [[243, 249, 210,  ..., 240, 242, 229],
         [231, 238, 203,  ..., 234, 243, 177],
         [206, 221, 167,  ..., 215, 206, 193],
         ...,
         [126, 111, 106,  ..., 112, 116, 131],
         [127, 140, 114,  ..., 127, 137, 130],
         [102, 111, 107,  ..., 145, 129, 139]],

        [[245, 250, 212,  ..., 244, 250, 238],
         [233, 240, 205,  ..., 239, 251, 186],
         [208, 222, 169,  ..., 221, 215, 202],
         ...,
         [137, 122, 117,  ..., 123, 127, 142],
         [138, 151, 125,  ..., 138, 148, 141],
         [113, 122, 118,  ..., 156, 140, 150]]], dtype=torch.uint8), 0.5), (tensor([[[251, 251, 251,  ..., 251, 251, 251],
         [251, 251, 25

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 3, 78, 200])
torch.Size([1, 3, 37, 98])
torch.Size([1, 3, 17, 47])
torch.Size([1, 3, 8, 23])
torch.Size([1, 3, 6, 21])
torch.Size([1, 3, 4, 19])
torch.Size([1, 228])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x228 and 76x1000)